In [2]:
#! pip install pytorch-pretrained-bert

In [3]:
import pandas as pd

In [17]:
raw_train_df = pd.read_csv('data/train_raw.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False, skiprows=1, header=None)
raw_train_df.head()

,0,1,2,3,4,5,6
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [18]:
raw_train_df.shape

(161297, 7)

In [19]:
raw_test_df = pd.read_csv('data/test_raw.tsv', delimiter='\t',encoding='utf-8', error_bad_lines=False, skiprows=1, header=None)
raw_test_df.head()

,0,1,2,3,4,5,6
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4


In [20]:
raw_test_df.shape

(53766, 7)

In [21]:
train_df = raw_train_df[[3,4]]

In [22]:
train_df.shape

(161297, 2)

In [23]:
train_df.head()

,3,4
0,"""It has no side effect, I take it in combinati...",9.0
1,"""My son is halfway through his fourth week of ...",8.0
2,"""I used to take another oral contraceptive, wh...",5.0
3,"""This is my first time using any form of birth...",8.0
4,"""Suboxone has completely turned my life around...",9.0


In [24]:
test_df = raw_test_df[[3,4]]

In [26]:
test_df.shape

(53766, 2)

In [29]:
test_df.columns = range(test_df.shape[1])

In [32]:
train_df.columns = range(train_df.shape[1])

In [33]:
train_df.head()

,0,1
0,"""It has no side effect, I take it in combinati...",9.0
1,"""My son is halfway through his fourth week of ...",8.0
2,"""I used to take another oral contraceptive, wh...",5.0
3,"""This is my first time using any form of birth...",8.0
4,"""Suboxone has completely turned my life around...",9.0


In [34]:
test_df.head()

,0,1
0,"""I&#039;ve tried a few antidepressants over th...",10.0
1,"""My son has Crohn&#039;s disease and has done ...",8.0
2,"""Quick reduction of symptoms""",9.0
3,"""Contrave combines drugs that were used for al...",9.0
4,"""I have been on this birth control for one cyc...",9.0


In [36]:
train_df[1] = (train_df[1] > 6.0).astype(int)
test_df[1] = (test_df[1] >  6.0).astype(int)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [37]:
train_df.head()

,0,1
0,"""It has no side effect, I take it in combinati...",1
1,"""My son is halfway through his fourth week of ...",1
2,"""I used to take another oral contraceptive, wh...",0
3,"""This is my first time using any form of birth...",1
4,"""Suboxone has completely turned my life around...",1


In [38]:
test_df.head()

,0,1
0,"""I&#039;ve tried a few antidepressants over th...",1
1,"""My son has Crohn&#039;s disease and has done ...",1
2,"""Quick reduction of symptoms""",1
3,"""Contrave combines drugs that were used for al...",1
4,"""I have been on this birth control for one cyc...",1


In [91]:
# BERT, however, wants data to be in a tsv file with a specific format as given below (Four columns, and no header row).
# Column 0: An ID for the row
# Column 1: The label for the row (should be an int)
# Column 2: A column of the same letter for all rows. BERT wants this so we’ll give it, but we don’t have a use for it.
# Column 3: The text for the row


train_df_bert = pd.DataFrame({
    'id':range(len(train_df)),
    'label':train_df[1],
    'alpha':['a']*train_df.shape[0],
    'text': train_df[0].replace(r'\n', ' ', regex=True).replace(r'"', ' ', regex=True).replace(r'\r', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
0,0,1,a,"It has no side effect, I take it in combinati..."
1,1,1,a,My son is halfway through his fourth week of ...
2,2,0,a,"I used to take another oral contraceptive, wh..."
3,3,1,a,This is my first time using any form of birth...
4,4,1,a,Suboxone has completely turned my life around...


In [92]:
# import re
# dev_df_bert = pd.DataFrame({
#     'id':range(len(test_df)),
#     'label':test_df[1],
#     'alpha':['a']*test_df.shape[0],
#     'text': str(test_df[0]).replace('\r\n"', ' ')#test_df[0].replace(r'\n', ' ', regex=True).replace(r'"', ' ', regex=True).replace(r'\r', ' ', regex=True)
# })

# dev_df_bert.head()

In [93]:
dev_df_bert = pd.DataFrame({
    'id':range(len(test_df)),
    'label':test_df[1],
    'alpha':['a']*test_df.shape[0],
    'text': test_df[0].replace(r'\n', ' ', regex=True).replace(r'"', ' ', regex=True).replace(r'\r', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
0,0,1,a,I&#039;ve tried a few antidepressants over th...
1,1,1,a,My son has Crohn&#039;s disease and has done ...
2,2,1,a,Quick reduction of symptoms
3,3,1,a,Contrave combines drugs that were used for al...
4,4,1,a,I have been on this birth control for one cyc...


In [95]:
train_df_bert.to_csv('data/train.tsv', sep='\t', index=False, header=False, line_terminator='\r\n')

In [97]:

dev_df_bert.to_csv('data/dev.tsv', sep='\t', index=False, header=False, line_terminator='\r\n')

In [98]:
train_df_bert['text'].head()

0     It has no side effect, I take it in combinati...
1     My son is halfway through his fourth week of ...
2     I used to take another oral contraceptive, wh...
3     This is my first time using any form of birth...
4     Suboxone has completely turned my life around...
Name: text, dtype: object